In [ ]:
%config InlineBackend.figure_formats = ["svg"]
import base64
import io
import json
import requests

import pandas as pd; pd.set_option('display.max_rows', 500)
import geopandas as gpd

In [ ]:
raw_data = requests.post(
    "https://atbackend.sipri.org/api/p/trades/trade-register-csv/",
    json={"filters": []},
).json()

In [ ]:
csv_lines = base64.b64decode(raw_data["bytes"]).decode("iso-8859-1").split("\n")
csv_lines[:15]

["Transfers of major conventional arms from All countries  to All countries . Deals with deliveries made for the year range 'Not specified' to 'Not specified' ",
 "A '?' in a column indicates uncertain data. The 'Number delivered' and the 'Year(s) of deliveries' refer only to deliveries in the selected year(s).",
 "An empty field for 'Number ordered' indicates that data is not yet available.",
 'SIPRI trend-indicator values (TIVs) are in millions.',
 "An empty field for 'SIPRI TIV for total order' indicates that data (on the number ordered and/or the TIV per unit) is not available.",
 "A '0' for 'SIPRI TIV of delivered weapons' indicates that the volume of deliveries is between 0 and 0.5 million SIPRI TIV; and an empty field indicates that no deliveries have been identified.",
 'Figures may not add up to stated totals due to the conventions of rounding.',
 'For the method used for the SIPRI TIV and explanations of the conventions; abbreviations and acronyms see <https://www.sipri.org/d

In [ ]:
first_line_index = next(idx for idx, line in enumerate(csv_lines) if line.startswith("Recipient,"))
first_line_index

11

In [ ]:
arms_df = pd.read_csv(io.StringIO("\n".join(csv_lines[first_line_index:])))
arms_df

,Recipient,Supplier,Year of order,,Number ordered,.1,Weapon designation,Weapon description,Number delivered,.2,Year(s) of delivery,status,Comments,SIPRI TIV per unit,SIPRI TIV for total order,SIPRI TIV of delivered weapons
0,Afghanistan,Brazil,2017.0,NaN,6.0,NaN,EMB-314 Super Tucano,trainer/combat aircraft,6.0,?,2018,New,A-29B version; financed by USA; ordered via US...,4.50,27.0,27.0
1,Afghanistan,Russia,2004.0,NaN,6.0,NaN,aircraft engine,aircraft engine,6.0,NaN,2005,New,Klimov TV-3-117 turboshaft; spare engines for ...,0.50,3.0,3.0
2,Afghanistan,Soviet Union,1977.0,?,26.0,NaN,AI-25,turbofan,26.0,?,1977; 1978,New,For 26 L-39 trainer aircraft from Czechoslovak...,0.60,15.6,15.6
3,Afghanistan,Soviet Union,1988.0,?,2300.0,?,R-17 Elbrus,SSM,2300.0,?,1988; 1989; 1990; 1991,New,Mainly for use against Mujahideen rebel forces,1.25,2875.0,2875.0
4,Afghanistan,United Kingdom,2009.0,NaN,2.0,NaN,Mi-17,transport helicopter,2.0,NaN,2010,Second hand,Second-hand Mi-17 version bought by UK for 'Pr...,2.90,5.8,5.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29053,Zimbabwe,Russia,1998.0,?,6.0,NaN,Mi-24P/Mi-35P,combat helicopter,6.0,NaN,1999,Second hand,Second-hand; bought for use in DRC against reb...,3.60,21.6,21.6
29054,Zimbabwe,Italy,1967.0,?,20.0,?,Model-56 105mm,towed gun,20.0,?,1967,New,Supplier uncertain,0.45,9.0,9.0
29055,Zimbabwe,Soviet Union,1975.0,?,15.0,?,T-34-85,tank,15.0,?,1975,Second hand,Second-hand; supplier uncertain,0.38,5.7,5.7
29056,Zimbabwe,China,2004.0,?,5.0,?,Type-89/ZSD-89,APC,5.0,?,2004,New,ARV version,0.30,1.5,1.5


For more on the SIPRI TIV (trend indicator value), see the SIPRI fact sheet ["Measuring International Arms Transfers"](https://www.sipri.org/sites/default/files/files/FS/SIPRIFS1212.pdf).

In [ ]:
arms_df.groupby(["Recipient", "Supplier", "Year of order"]).count()

Number ordered   .1  \
Recipient            Supplier           Year of order                           
ANC (South Africa)*  Soviet Union       1988.0         0               1    1   
Afghanistan          Belarus            2018.0         0               1    0   
                     Bosnia-Herzegovina 2009.0         0               1    0   
                                        2012.0         1               1    1   
                     Brazil             2013.0         0               1    0   
...                                                   ..             ...  ...   
unknown recipient(s) United States      1995.0         0               1    0   
                                        2010.0         0               1    1   
                                        2013.0         0               1    1   
                                        2018.0         2               2    2   
                                        2023.0         1               1    0   

                                                       Weapon designation  \
Recipient            Supplier           Year of order                       
ANC (South Africa)*  Soviet Union       1988.0                          1   
Afghanistan          Belarus            2018.0                          1   
                     Bosnia-Herzegovina 2009.0                          1   
                                        2012.0                          1   
                     Brazil             2013.0                          1   
...                                                                   ...   
unknown recipient(s) United States      1995.0                          1   
                                        2010.0                          1   
                                        2013.0                          1   
                                        2018.0                          2   
                                        2023.0                          2   

                                                       Weapon description  \
Recipient            Supplier           Year of order                       
ANC (South Africa)*  Soviet Union       1988.0                          1   
Afghanistan          Belarus            2018.0                          1   
                     Bosnia-Herzegovina 2009.0                          1   
                                        2012.0                          1   
                     Brazil             2013.0                          1   
...                                                                   ...   
unknown recipient(s) United States      1995.0                          1   
                                        2010.0                          1   
                                        2013.0                          1   
                                        2018.0                          2   
                                        2023.0                          2   

                                                       Number delivered   .2  \
Recipient            Supplier           Year of order                          
ANC (South Africa)*  Soviet Union       1988.0                        1    1   
Afghanistan          Belarus            2018.0                        1    0   
                     Bosnia-Herzegovina 2009.0                        1    1   
                                        2012.0                        1    0   
                     Brazil             2013.0                        1    1   
...                                                                 ...  ...   
unknown recipient(s) United States      1995.0                        1    0   
                                        2010.0                        1    1   
                                        2013.0                        1    1   
                                        2018.0                        2    2   
                                        

In [ ]:
# arms_df[(arms_df["Weapon designation"] == "Mi-17") & (arms_df["Supplier"] != "Russia") & (arms_df["Supplier"] != "Soviet Union")]
arms_df[arms_df["status"] == "Second hand"]

,Recipient,Supplier,Year of order,,Number ordered,.1,Weapon designation,Weapon description,Number delivered,.2,Year(s) of delivery,status,Comments,SIPRI TIV per unit,SIPRI TIV for total order,SIPRI TIV of delivered weapons
4,Afghanistan,United Kingdom,2009.0,NaN,2.0,NaN,Mi-17,transport helicopter,2.0,NaN,2010,Second hand,Second-hand Mi-17 version bought by UK for 'Pr...,2.90,5.8,5.8
7,Afghanistan,Soviet Union,1987.0,?,40.0,?,MiG-21MF,fighter aircraft,40.0,?,1987; 1988; 1989; 1990,Second hand,Second-hand; probably incl some MiG-21UM,4.28,171.2,171.2
11,Afghanistan,Soviet Union,1970.0,?,50.0,?,BRDM-1,reconnaissance AV,50.0,?,1970; 1971,Second hand,Probably second-hand,0.05,2.4,2.4
12,Afghanistan,Soviet Union,1979.0,?,250.0,?,BRDM-2,reconnaissance AV,250.0,?,1979; 1980; 1981; 1982; 1983; 1984; 1985; 1986...,Second hand,Probably second-hand,0.06,15.0,15.0
16,Afghanistan,Soviet Union,1989.0,?,40.0,?,MiG-21bis,fighter aircraft,40.0,?,1989; 1990,Second hand,Second-hand,5.00,200.0,200.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29049,Zimbabwe,Mozambique,1976.0,?,5.0,NaN,BN-2 Islander,light transport aircraft,5.0,?,1977,Second hand,Second-hand,0.24,1.2,1.2
29050,Zimbabwe,Kenya,1981.0,NaN,5.0,NaN,Hunter,FGA aircraft,5.0,NaN,1981,Second hand,Second-hand; sold via UK company; Hunter FGA-8...,1.60,8.0,8.0
29052,Zimbabwe,Ukraine,2000.0,?,1.0,?,An-12,transport aircraft,1.0,?,2001,Second hand,Second-hand,9.20,9.2,9.2
29053,Zimbabwe,Russia,1998.0,?,6.0,NaN,Mi-24P/Mi-35P,combat helicopter,6.0,NaN,1999,Second hand,Second-hand; bought for use in DRC against reb...,3.60,21.6,21.6


In [ ]:
arms_df["Weapon designation"].value_counts()

,count
Weapon designation,
aircraft engine,417
vehicle engine,398
ship engine,379
Mi-17,239
M-113,225
...,...
F-14A Tomcat,1
Kharg,1
Tigercat-2,1


In [ ]:
capitals_map = {
    "ANC (South Africa)*": "South Africa",
    "Anti-Castro rebels (Cuba)*": "Cuba",
    "Amal (Lebanon)*": "Lebanon",
    "Armas (Guatemala)*": "Guatemala",
    "Contras (Nicaragua)*": "Nicaragua",
    "Darfur rebels (Sudan)*": "Sudan",
    "ELF (Ethiopia)*": "Ethiopia",
    "EPLF (Ethiopia)*": "Ethiopia",
    "FRELIMO (Portugal)*": "Portugal",
    "Haiti rebels*": "Haiti",
    "Hezbollah (Lebanon)*": "Lebanon",
    "Houthi rebels (Yemen)*": "Yemen",
    "Indonesia rebels*": "Indonesia",
    "Khmer Rouge (Cambodia)*": "Cambodia",
    "Kurdistan Regional Government (Iraq)*": "Iraq",
    "LF (Lebanon)*": "Lebanon",
    "LRA (Uganda)*": "Uganda",
    "LTTE (Sri Lanka)*": "Sri Lanka",
    "Libya GNC": "Libya",
    "Libya HoR*": "Libya",
    "Congo": "Congo (Brazzaville)",
    "DR Congo": "Congo (Kinshasa)",
    "MNLF (Philippines)*": "Philippines",
    "MPLA (Portugal)*": "Portugal",
    "MTA (Myanmar)*": "Myanmar",
    "Micronesia": "Federated States of Micronesia",
    "Mujahedin (Afghanistan)*": "Afghanistan",
    "NLA (Macedonia)*": "North Macedonia",
    "NTC (Libya)*": "Libya",
    "Northern Alliance (Afghanistan)*": "Afghanistan",
    "Northern Cyprus": "Cyprus",
    "PAIGC (Portugal)*": "Portugal",
    "PIJ (Israel/Palestine)*": "Israel",
    "PKK (Turkiye)*": "Turkey",
    "PLO (Israel)*": "Israel",
    "PRC (Israel/Palestine)*": "Israel",
    "Pathet Lao (Laos)*": "Laos",
    "Provisional IRA (UK)*": "United Kingdom",
    "RPF (Rwanda)*": "Rwanda",
    "RUF (Sierra Leone)*": "United Kingdom",
    "SLA (Lebanon)*": "Lebanon",
    "SNA (Somalia)*": "Somalia",
    "SPLA (Sudan)*": "Sudan",
    "Southern rebels (Yemen)*": "Yemen",
    "Syria rebels*": "Syria",
    "Turkiye": "Turkey",
    "UAE": "United Arab Emirates",
    "UIC (Somalia)*": "Somalia",
    "UNITA (Angola)*": "Angola",
    "Ukraine Rebels*": "Ukraine",
    "United States": "United States of America",
    "United Wa State (Myanmar)*": "Myanmar",
    "Viet Minh (France)*": "France",
    "Viet Nam": "Vietnam",
    "ZAPU (Zimbabwe)*": "Zimbabwe",
    "GUNT (Chad)*": "Chad",
    "FAN (Chad)*": "Chad",
    "FMLN (El Salvador)*": "El Salvador",
    "Gambia": "The Gambia",
    "Lebanon Palestinian rebels*": "Lebanon",
    "Cote d'Ivoire": "Ivory Coast",
    "Bahamas": "The Bahamas",
    "FNLA (Angola)*": "Angola",
    "Cabo Verde": "Cape Verde",
    "Timor-Leste": "East Timor",
    "Saint Vincent": "Saint Vincent and the Grenadines",
    "South Vietnam": "Vietnam",  # Saigon is now Ho Chi Minh City
    "Viet Cong (South Vietnam)*": "Vietnam",
    "Soviet Union": "Russia",
    "NATO**": "Belgium",  # NATO headquarters in Brussels
    'European Union**': "Belgium",  # EU headquarters in Brussels
    "OSCE**": "Austria",  # secretariat in Vienna
    "Yemen Arab Republic (North Yemen)": "Yemen",  # same capital as Yemen (Sanaa)
    "North Yemen": "Yemen",  # same capital as Yemen (Sanaa)
    "Czechoslovakia": "Czechia",  # same capital as the modern Czech Republic (Prague)
    "Yugoslavia": "Serbia",  # same capital as Serbia (Belgrade)
}

exclude_flows = {
    "nan",
    "unknown rebel group*",
    "unknown recipient(s)",
    'unknown supplier(s)',
    "United Nations**",
    "Regional Security System**",
    "African Union**",
    # TODO
    "Biafra",
    "Bosnia-Herzegovina",
    "East Germany (GDR)",
    "Guinea-Bissau",
    "Hamas (Palestine)*",
    "Katanga",
    "Palestine",
    "South Yemen",
    '0.25',
    '3',
    'Aruba',
    "South Sudan",
    'Western Sahara'
}


In [ ]:
places_gdf = gpd.read_file("https://naciscdn.org/naturalearth/110m/cultural/ne_110m_populated_places_simple.zip")
capitals_gdf = places_gdf[places_gdf["adm0cap"] == 1].set_index("adm0name")
# force each nation to have exactly one capital
capitals_gdf = capitals_gdf[~capitals_gdf.index.isin(["Sucre", "Yamoussoukro", "Bloemfontein", "Pretoria"])]

In [ ]:
capitals_gdf[["latitude", "longitude"]].head(10)

,latitude,longitude
adm0name,,
Vatican,41.903282,12.453387
San Marino,43.936096,12.441770
Liechtenstein,47.133724,9.516670
Luxembourg,49.611660,6.130003
Federated States of Micronesia,6.916644,158.149974
Marshall Islands,7.103004,171.380000
Tuvalu,-8.516652,179.216647
Palau,7.487396,134.626549
Monaco,43.739646,7.406913


ideas:
* pick a protracted conflict (e.g. Israel–Palestine)
* pick a year or two
* analyze the immediate effects of the collapse of the USSR on arms flows
* analyze secondhand arms sales (in the context of the Cold War?) – how are arms recycled?
* analyze the popularity and regionality of a particular arm (e.g. the Mi-17) over time

## Advanced flow visualizations with FlowmapBlue (Colab only)

In [ ]:
!pip install -q gspread_pandas

In [ ]:
from google.colab import auth
import gspread_pandas
from google.auth import default

In [ ]:
auth.authenticate_user()
creds, _ = default()

In [ ]:
df = pd.DataFrame([1,2,3,4], columns=['T'])
##create new sheet and dump dataframe
spread = gspread_pandas.Spread('PUBPOL 2130: PUMS migration flows', creds=creds, create_spread=True)
spread.df_to_sheet(df, index=False, headers=True, start='A1',)

In [ ]:
flowmap_arms_df = arms_df[~arms_df["Supplier"].isin(exclude_flows) & ~arms_df["Recipient"].isin(exclude_flows)].rename(
    columns={
        "Year of order": "order_year",
        "Recipient": "recipient",
        "Supplier": "supplier",
        "SIPRI TIV for total order": "order_sipri_tiv"
    }
)
flowmap_arms_df["order_year"] = flowmap_arms_df["order_year"].astype(int)
flowmap_arms_df = flowmap_arms_df[flowmap_arms_df["order_year"] >= 1950]

In [ ]:
orders_by_year_df = flowmap_arms_df.groupby(["order_year", "recipient", "supplier"]).sum()["order_sipri_tiv"]
orders_by_year_df

order_year  recipient  supplier      
1950        Argentina  United States     135.51
            Australia  United Kingdom    337.68
                       United States       0.60
            Belgium    United Kingdom     78.00
                       United States     756.12
                                          ...  
2023        Uruguay    Spain               0.00
                       United States      12.72
            Viet Nam   India              30.00
            Zambia     United States      20.80
            Zimbabwe   China              20.40
Name: order_sipri_tiv, Length: 15054, dtype: float64

In [ ]:
flowmap_locations_df = pd.DataFrame(
    [
        {
            "id": loc,
            "name": loc,
            "lat": capitals_gdf.loc[capitals_map.get(loc, loc), "latitude"],
            "lon": capitals_gdf.loc[capitals_map.get(loc, loc), "longitude"],
        }
        for loc in set(flowmap_arms_df["Supplier"]) | set(flowmap_arms_df["Recipient"])
    ]
)
flowmap_locations_df

,id,name,lat,lon
0,Thailand,Thailand,13.751945,100.514699
1,Libya GNC,Libya GNC,32.8925,13.180012
2,Viet Nam,Viet Nam,21.035273,105.848068
3,Congo,Congo,-4.25724,15.282744
4,Southern rebels (Yemen)*,Southern rebels (Yemen)*,15.356679,44.204648
5,Nigeria,Nigeria,9.085279,7.531382
6,Haiti rebels*,Haiti rebels*,18.542971,-72.33798
7,Estonia,Estonia,59.433877,24.728041
8,Cuba,Cuba,23.133905,-82.366128
9,Indonesia,Indonesia,-6.172472,106.827492


In [ ]:
capitals_gdf.loc["France", "geometry"].coords.xy[1][0]

48.85809231626911

In [ ]:
def generate_arms_flow(
    sheet_creds,
    locations_df,
    sheet_title: str,
    flows: dict[str, pd.DataFrame],
    flow_title: str = "",
    description: str = "",
    created_by_name: str = "",
    created_by_email: str = "",
):
    props_df = pd.DataFrame([
        {'property': 'title', 'value': flow_title},
        {'property': 'description', 'value': description},
        {'property': 'source.name', 'value': 'SIPRI Arms Transfers Database'},
        {'property': 'source.url', 'value': 'https://www.sipri.org/databases/armstransfers'},
        {'property': 'createdBy.name', 'value': created_by_name},
        {'property': 'createdBy.email', 'value': created_by_email},
        {'property': 'colors.scheme', 'value': 'SunsetDark'},
        {'property': 'colors.darkMode', 'value': 'yes'},
        {'property': 'animate.flows', 'value': 'yes'},
        {'property': 'clustering', 'value': 'yes'},
        {'property': 'flows.sheets', 'value': ','.join(flows)},  # TODO: force plaintext cell type
        {'property': 'msg.locationTooltip.incoming', 'value': 'Inbound arms transfers (TIV)'},
        {'property': 'msg.locationTooltip.outgoing', 'value': 'Outbound arms transfers (TIV)'},
        {'property': 'msg.flowTooltip.numOfTrips', 'value': 'Arms transfer (TIV)'},
        {'property': 'msg.totalCount.allTrips', 'value': '{0} TIVs'},
        {'property': 'msg.totalCount.countOfTrips', 'value': '{0} of {1} TIVs'}
    ])
    props_spread = gspread_pandas.Spread(
        sheet_title,
        "properties",
        creds=sheet_creds,
        create_spread=True,
        create_sheet=True,
    )
    props_spread.df_to_sheet(props_df, index=False, headers=True, start='A1',)

    locations_spread = gspread_pandas.Spread(
        sheet_title,
        "locations",
        creds=sheet_creds,
        create_sheet=True,
    )
    locations_spread.df_to_sheet(locations_df, index=False, headers=True, start='A1',)

    for key, df in flows.items():
        print(key)
        flow_spread = gspread_pandas.Spread(
            sheet_title,
            key,
            creds=sheet_creds,
            create_sheet=True,
        )
        flow_spread.df_to_sheet(df, index=False, headers=True, start='A1',)

In [ ]:
generate_arms_flow(
    sheet_creds=creds,
    locations_df=flowmap_locations_df,
    sheet_title="PUBPOL 2130: arms transfers (orders over time)",
    flow_title="SIPRI Arms Transfers Database: orders over time",
    flows={
        str(year): year_df.reset_index().rename(columns={
            "supplier": "origin",
            "recipient": "dest",
            "order_sipri_tiv": "count",
        })
        for year, year_df in orders_by_year_df.groupby(level=0)
    }
)

1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
